# Eclipse HDR timelapse

If you have a webcam or USB camera connected to a PC in the path of totality, then point your camera towards the sky and run this notebook.

This notebook will:
- Access you camera with open-cv.
- Determine timepoints of interest to capture the duration of the eclipse at 1-minute intervals. 
- For each timepoint, will capture a user-defined range of exposures, that can be used to create an HDR image.

In [1]:
# Import modules
import cv2
from camera_automation import *

All the setings you might change are in the next cell:

This notebook allows you to do a dry run of your acquisition before the actual eclipse by setting 
TEST_RUN = True
Which will set your timelapse timepoints to todays date
and choosing a TEST_OFFSET 
which will offset the starting hour of the acquisition by a chosen number of hours.

For example, if I want to test the exact eclipse timecourse today, I would set TEST_OFFSET to 0.

Otherwise, if I wanted to test it tonight when the sun is down, I would set TEST_OFFSET to 8, to offset by 8 hours.

In [2]:
# Define your parameters:
TEST_RUN = True
TEST_OFFSET = -3.5 #h
TIMELAPSE_INTERVAL = 60 # seconds
YEAR = "2024"
BASE_OUTPUT_FOLDER = 'M:/cv2_output/'
EXPOSURE_BASES = list(range(-11, 1)) 

exposure_times = [2 ** base for base in EXPOSURE_BASES]
formatted_exposure_times = [f"{time:.5f}" if time < 1 else str(int(time)) for time in exposure_times]
print('Camera exposures:')
formatted_exposure_times




Camera exposures:


['0.00049',
 '0.00098',
 '0.00195',
 '0.00391',
 '0.00781',
 '0.01562',
 '0.03125',
 '0.06250',
 '0.12500',
 '0.25000',
 '0.50000',
 '1']

In [3]:
# Initialize camera
camera = cv2.VideoCapture(0)

Find your location on the map at:
[https://www.timeanddate.com/eclipse/map/2024-april-8](https://www.timeanddate.com/eclipse/map/2024-april-8)

And copy the text, for example:

- **Partial begins:** Apr 8 at 2:14:31 pm
- **Full begins:** Apr 8 at 3:27:10 pm
- **Maximum:** Apr 8 at 3:27:34 pm
- **Full ends:** Apr 8 at 3:27:58 pm
- **Partial ends:** Apr 8 at 4:36:52 pm


In [4]:
# Paste the text below in the triple quotation marks
ECLIPSE_TEXT = """
Partial begins
Apr 8 at 2:14:31 pm
Full begins
Apr 8 at 3:27:10 pm
Maximum
Apr 8 at 3:27:34 pm
Full ends
Apr 8 at 3:27:58 pm
Partial ends
Apr 8 at 4:36:52 pm
"""

Extract the formatted datetimes from the text, and verify that it matches what is seen on the webpage. 
Especially important is the middle value that is the maximum.
These are the timepoints you will use to automate your camera. 

In [5]:
# Extract datetime objects

extracted_datetimes = extract_datetime(ECLIPSE_TEXT, YEAR)
for dt in extracted_datetimes:
    print(dt)


2024-04-08 14:14:31
2024-04-08 15:27:10
2024-04-08 15:27:34
2024-04-08 15:27:58
2024-04-08 16:36:52


Since we want the timelapse to capture the entire duration of the eclipse, and not just selected timepoints, then we will create a list of timepoints to use. 

This will ensure that there is an acquisition started exactly at the time of the maximum, by centering the timepoints at specifically this time. 

In [6]:
if TEST_RUN:
    
    # Change each of the datetime objects to be for today (FOR TESTING)
    today = datetime.now().date()
    datetimes_for_today = [datetime.combine(today, dt.time()) for dt in extracted_datetimes]

    # Offset the timepoints around the eclipse a user-defined amount
    adjusted_timepoints = [tp + timedelta(hours=TEST_OFFSET) for tp in datetimes_for_today]

    # Now create 1min intervals centered on the middle one.
    centered_timepoints = generate_centered_timepoints(adjusted_timepoints, TIMELAPSE_INTERVAL)


# Otherwise use the real eclipse date and time
else:

    # Now create 1min intervals centered on the middle one.
    centered_timepoints = generate_centered_timepoints(extracted_datetimes, TIMELAPSE_INTERVAL)


# Add a timepoint for 1 minute from now at the start to test the aquisition is running properly
one_minute_from_now = datetime.now() + timedelta(minutes=1)
if one_minute_from_now < centered_timepoints[0]:
    centered_timepoints.insert(0, one_minute_from_now)

# Print the list of all timepoints
for tp in centered_timepoints:
    print(tp)

2024-04-05 10:24:43.685018
2024-04-05 10:44:31
2024-04-05 10:46:34
2024-04-05 10:47:34
2024-04-05 10:48:34
2024-04-05 10:49:34
2024-04-05 10:50:34
2024-04-05 10:51:34
2024-04-05 10:52:34
2024-04-05 10:53:34
2024-04-05 10:54:34
2024-04-05 10:55:34
2024-04-05 10:56:34
2024-04-05 10:57:34
2024-04-05 10:58:34
2024-04-05 10:59:34
2024-04-05 11:00:34
2024-04-05 11:01:34
2024-04-05 11:02:34
2024-04-05 11:03:34
2024-04-05 11:04:34
2024-04-05 11:05:34
2024-04-05 11:06:34
2024-04-05 11:07:34
2024-04-05 11:08:34
2024-04-05 11:09:34
2024-04-05 11:10:34
2024-04-05 11:11:34
2024-04-05 11:12:34
2024-04-05 11:13:34
2024-04-05 11:14:34
2024-04-05 11:15:34
2024-04-05 11:16:34
2024-04-05 11:17:34
2024-04-05 11:18:34
2024-04-05 11:19:34
2024-04-05 11:20:34
2024-04-05 11:21:34
2024-04-05 11:22:34
2024-04-05 11:23:34
2024-04-05 11:24:34
2024-04-05 11:25:34
2024-04-05 11:26:34
2024-04-05 11:27:34
2024-04-05 11:28:34
2024-04-05 11:29:34
2024-04-05 11:30:34
2024-04-05 11:31:34
2024-04-05 11:32:34
2024-04-05 11

You can verify the list of timepoints looks correct and covers the range of the eclipse at your location. 
The first timepoint is always 1 minute from now, so we can test the acquisition. 
For some reason, occasionally connecting to the camera will fail, so by making the first timepoint of the acquisition exactly 1 minute in the future, 
you can verify the automation is working correctly before leaving to see the eclipse. 




In [7]:
if camera.isOpened():

    timed_capture(camera, BASE_OUTPUT_FOLDER, EXPOSURE_BASES, centered_timepoints, label='eclipse')
    # camera.release()
else:
    print("Camera could not be initialized.")

Created folder M:/cv2_output/eclipse_2024-04-05_10-24-43
Starting acquisition: 2024-04-05_10-24-43
All images successfully captured for timepoint: 2024-04-05_10-24-43
